In [ ]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler as scaler

color_0_1 = ('#40DFEF', '#E78EA9')

pesticides=pd.read_csv("N:\\Machine learning\\Yield production\\pesticides.csv")
rainfall=pd.read_csv("N:\\Machine learning\\Yield production\\rainfall.csv")
temperature=pd.read_csv("N:\\Machine learning\\Yield production\\temp.csv")
yield_data=pd.read_csv("N:\\Machine learning\\Yield production\\yield.csv")


yield_df=pd.read_csv("N:\\Machine learning\\Yield production\\yield_df.csv")


yield_df.rename({'Area':'Country','hg/ha_yield':'Yield (hg/ha)','average_rain_fall_mm_per_year':'Rainfall (mm)'},axis=1,inplace=True)
yield_df.drop('Unnamed: 0',axis=1,inplace=True)


# correlation matrix
corrdata=yield_df.drop(['Year','Yield (hg/ha)'],axis=1)

plt.figure(figsize=(4,4))

sns.heatmap(corrdata.corr(), cmap='Blues', annot=True)
plt.show()


le=LabelEncoder()

yield_df['Country_encoded']=le.fit_transform(yield_df['Country'])
yield_df['Item_encoded']=le.fit_transform(yield_df['Item'])

yield_df.drop(['Country','Item'],axis=1,inplace=True)


# Boxplot before outlier removal
plt.figure(figsize=(20,3))

plt.subplot(1,2,1)
sns.boxplot(x=yield_df['Yield (hg/ha)'])
plt.title('Box Plot of Yield before removing outliers')

for i in yield_df.columns:
    q75, q25 = np.percentile(yield_df[i], [75 ,25])
    iqr = q75 - q25
    min_val = q25 - (iqr*1.5)
    max_val = q75 + (iqr*1.5)
    
    
    yield_df=yield_df[(yield_df[i]<max_val)]
    yield_df=yield_df[(yield_df[i]>min_val)]


numcols=['Rainfall (mm)','pesticides_tonnes','avg_temp']


# boxplot after outlier removal

plt.subplot(1,2,2)
sns.boxplot(x=yield_df['Yield (hg/ha)'])
plt.title('Box Plot of Yield after removing outliers')
    
plt.show()


Y=yield_df['Yield (hg/ha)']
X=yield_df.drop('Yield (hg/ha)',axis=1)


min_max_scaler=MinMaxScaler()
X_scaled=pd.DataFrame(min_max_scaler.fit_transform(X))



X_scaled.columns=X.columns



x_train,x_test,y_train,y_test=train_test_split(X_scaled,Y,test_size=0.2,random_state=42)

# print(x_train.shape)

# 19753,6

neural_regressor = tf.keras.models.Sequential()


neural_regressor.add(tf.keras.layers.Dense(units=12, activation='selu',))
neural_regressor.add(tf.keras.layers.Dense(units=12, activation='selu'))
# neural_regressor.add(tf.keras.layers.Dense(units=12, activation='selu'))
neural_regressor.add(tf.keras.layers.Dense(units=1, activation='linear'))



# Default value of learning rate is 0.001
neural_regressor.compile(optimizer=tf.keras.optimizers.Adam(0.1), loss='mse',metrics=['mean_absolute_error'])

# 20 batches are used.
# batch_size = len(x_train) // 200
                                                  

plot_data = neural_regressor.fit(x_train,y_train,epochs=300,)


y_pred = neural_regressor.predict(x_test)
r2 = r2_score(y_test, y_pred)
print("R2 score: ", r2)
print("MSE : ",mean_squared_error(y_test, y_pred))







loss_train = plot_data.history['mean_absolute_error']


# Loss plot vs number of epochs

epochs = range(1,601)
plt.plot(epochs, loss_train, 'g', label='Training loss')

plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Model evaluation plot
ry=range(0,len(x_test))
plt.plot(ry, y_test,color='g') 

plt.plot(ry, y_pred,color='k') 



plt.show()